# Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the
[previous homework](../08-deep-learning/homework.md).

Download the model from here:

https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [1]:
# !wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [2]:
import os
import numpy as np
import tensorflow as tf
from keras.saving import load_model
from PIL import Image

2024-12-06 22:34:17.587328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 22:34:17.599638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 22:34:17.603026: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 22:34:17.612774: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-06 22:34:18.219723: W tensorflow/compiler/tf2

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

In [3]:
keras_model = load_model("model_2024_hairstyle.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()
with open("model_2024_hairstyle.tflite", "wb") as f:
    f.write(tflite_model)

I0000 00:00:1733495660.890500  142665 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733495660.924957  142665 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733495660.926808  142665 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733495660.929979  142665 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

INFO:tensorflow:Assets written to: /tmp/tmp4o855bmy/assets


INFO:tensorflow:Assets written to: /tmp/tmp4o855bmy/assets


Saved artifact at '/tmp/tmp4o855bmy'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  126531106185536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126531106190464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126531106518848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126531106515680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126531106520256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126531106506704: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1733495662.542985  142665 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1733495662.543000  142665 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-12-06 22:34:22.543333: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4o855bmy
2024-12-06 22:34:22.543804: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-06 22:34:22.543814: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp4o855bmy
2024-12-06 22:34:22.546594: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-12-06 22:34:22.547016: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-06 22:34:22.611895: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp4o855bmy
2024-12-06 22:34:22.617136: I tensorflow/cc/saved_model/loader.cc

In [4]:
size_in_mb = os.path.getsize("model_2024_hairstyle.tflite") / (1024 * 1024)
print(f"Model size: {round(size_in_mb)} MB")

Model size: 77 MB


## Question 2

To be able to use this model, we need to know the index of the input and
the index of the output.

What's the output index for this model?

In [5]:
interpreter = tf.lite.Interpreter(model_path="model_2024_hairstyle.tflite")
interpreter.allocate_tensors()
output_index = interpreter.get_output_details()[0]['index']
print(f"The output index for this model is {output_index}")

The output index for this model is 13


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Preparing the image

You'll need some code for downloading and resizing images. You can use
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image:

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

In [6]:
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    img = np.array(img, dtype=np.float32)
    img = np.expand_dims(img, axis=0) / 255.0
    return img

## Question 3

Now we need to turn the image into numpy array and pre-process it.

> Tip: Check the previous homework. What was the pre-processing
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

In [7]:
target_size = (200, 200)
img = Image.open("img1.jpeg")
img = prepare_image(img, target_size)
print(f"The value in the first pixel, the R channel is {round(img[0,0,0,0], 2)}")

The value in the first pixel, the R channel is 0.23999999463558197


## Question 4

Now let's apply this model to this image. What's the output of the model?

In [8]:
input_index = interpreter.get_input_details()[0]['index']
interpreter.set_tensor(input_index, img)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
print(f"The output of the model is {round(preds[0,0], 3)}")

The output of the model is 0.8939999938011169


## Prepare the lambda code

Now you need to copy all the code into a separate python file. You will
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook
by importing the file and invoking the function from this file.

## Docker

For the next two questions, we'll use a Docker image that we already
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10

COPY model_2024_hairstyle_v2.tflite .

RUN pip install numpy==1.23.1
```

Note that it uses Python 3.10. The latest models of TF Lite
do not support Python 3.12 yet, so we need to use 3.10. Also,
for this part, we will use TensorFlow 2.14.0. We have tested
it, and the models created with 2.17 could be served with 2.14.0.

For that image, we also needed to use an older version of numpy
(1.23.1)

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The wheel for this combination that you'll need to use in your Docker image is https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl

## Question 5

Download the base image `agrigorev/model-2024-hairstyle:v3`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [9]:
%%capture
!docker pull agrigorev/model-2024-hairstyle:v3

In [10]:
!docker images agrigorev/model-2024-hairstyle

REPOSITORY                       TAG       IMAGE ID       CREATED      SIZE
agrigorev/model-2024-hairstyle   v3        607850a2138e   2 days ago   782MB


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included.
The name of the file with the model is `model_2024_hairstyle_v2.tflite` and it's
in the current workdir in the image (see the Dockerfile above for the
reference).
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

In [11]:
%%capture
!docker build -t hairstyle-lambda .
!docker run --rm -d -p 8080:8080 hairstyle-lambda

In [12]:
!python test.py

The output from the model is 0.43


## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.

## Publishing to Docker hub

Just for the reference, this is how we published our image to Docker hub:

```bash
docker build -t model-2024-hairstyle -f homework.dockerfile .
docker tag model-2024-hairstyle:latest agrigorev/model-2024-hairstyle:v3
docker push agrigorev/model-2024-hairstyle:v3
```

(You don't need to execute this code)

## Submit the results

* Submit your results here: https://courses.datatalks.club/ml-zoomcamp-2024/homework/hw09
* If your answer doesn't match options exactly, select the closest one